# 2D flying-snake at Re=2000 and AoA=25deg (surface markers)

In [1]:
import os
import sys
import collections

import ipywidgets

import snake
from snake import miscellaneous
from snake.cuibm.simulation import CuIBMSimulation
from snake.ibamr.simulation import IBAMRSimulation
from snake.geometry import Geometry

In [2]:
print('Python version: ' + sys.version)
print('snake version:' + snake.__version__)

Python version: 2.7.11 |Anaconda 2.5.0 (64-bit)| (default, Dec  6 2015, 18:08:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
snake version:0.1.dev


---

### Objectives of the simulation

We aim to compute the flow around a snake cross-section with angle-of-attack 25 degrees at Reynolds number 2000.

We use the `ConstraintIBMethod` implemented in IBAMR.

We place the Lagrangian markers only on the immersed-boundary where the resolution is the same as the finest background mesh.

We use a "stabilized outlet" boundary condition to avoid flow propagation back into the computational domain. The stabilized condition was suggested by [Boyce Griffith](https://groups.google.com/forum/#!topic/ibamr-users/EEHzVxMyLWw) and acts as a force pushing the fluid outside the computational domain.

---

### General information

* software: `IBAMR` 
* version: SHA: `9464268fc0507fe8c94be126e6beaf267a466788`
* dependencies: `OpenMPI-1.8` / `GCC-4.7.0` / `SAMRAI-2.4.4` / `HDF5-1.8.13` / `SILO-4.10`
* linear algebra library: `PETSc-3.5.2`
* submission date: 2015/07/28 (mesnardo)
* running date: 2015/07/28
* machine: `colonial one` (short queue, 16 processes, 1 node)

In [ ]:
simulation = IBAMRSimulation(description='IBAMR')

---

### Computational domain and mesh

* Adaptive mesh refinement technique.
* domain: `[-15, 15]x[-15, 15]`
* `30x30` cells on the coarsest level (cell-width: 30.0/30=1.0).
* We are using 5 levels of refinement (counting level 0) with a refinement ratio of 4 between two consecutive levels (minimum cell-width: 1.0/4.0^4=0.004).
* Vorticity thresholds are used to tag cells for refinement.

---

### Boundary conditions

- x-velocity: 
    + inlet: fixed-value (1.0), 
    + outlet: zero-gradient, 
    + bottom/top: zero-gradient
- y-velocity:
    + inlet: fixed-value (0.0),
    + outlet: zero-gradient,
    + bottom/top: fixed-value (0.0)
- BoundaryStabilization of type "OUTFLOW"

---

### Simulation parameters

- start/end/saving/time-increment: 0 / 80 / max CFL=0.3
- INS: FGMRES (default, rtol 1.0E-06)
- preconditioner: projection method
    + velocity solver: CG (default, rtol 1.0E-06)
    + pressure solver: CG (default, rtol 1.0E-06)

---

### Input file

In [ ]:
input_info = os.path.join(simulation.directory, 'input2d')
%cat $input_info

---

### Creating the Lagrangian markers

In [ ]:
body = Geometry(file_path=os.path.join(simulation.directory,
                                       'flyingSnake2dAoA0ds0.004.bdy'))
body.rotation(pitch=-25.0)
body.write(file_path=os.path.join(simulation.directory,
                                  'flyingSnake2dAoA25ds004.vertex'))

### Running the case

To submit the simulation on `Colonial One`:
    > sbatch runIBAMRColonialOne.sh

---

### Post-processing

We computed 80 non-dimensional time-units in about 20 hours.

#### Forces

In [ ]:
simulation.read_forces()
simulation.get_mean_forces(limits=[32.0, 64.0])
simulation.get_strouhal(limits=[32.0, 64.0], order=200)
simulation.plot_forces(display_coefficients=True,
                       coefficient=-2.0,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       style='snakeReproducibility',
                       save_name='forceCoefficients')
dataframe = simulation.create_dataframe_forces(display_strouhal=True,
                                               display_coefficients=True,
                                               coefficient=-2.0)

In [ ]:
krishnan = CuIBMSimulation(description='Krishnan et al. (2014)')
krishnan.read_forces(file_path=os.path.join(os.environ['SNAKE'],
                                            'resources',
                                            'flyingSnake2d_cuibm_anush',
                                            'flyingSnake2dRe2000AoA25',
                                            'forces'))
krishnan.get_mean_forces(limits=[32.0, 64.0])
krishnan.get_strouhal(limits=[32.0, 64.0], order=200)
simulation.plot_forces(display_coefficients=True,
                       coefficient=-2.0,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       other_simulations=krishnan,
                       other_coefficients=2.0,
                       style='snakeReproducibility',
                       save_name='forceCoefficientsCompareKrishnanEtAl2014')
dataframe2 = krishnan.create_dataframe_forces(display_strouhal=True,
                                              display_coefficients=True,
                                              coefficient=2.0)

In [ ]:
dataframes = dataframe.append(dataframe2)
print(dataframes)

In [ ]:
images_directory = os.path.join(simulation.directory,
                                'images')
figures = collections.OrderedDict()
figures['present'] = os.path.join(images_directory,
                                  'forceCoefficients.png')
figures['present + Krishnan et al. (2014)'] = os.path.join(images_directory,
                                                           'forceCoefficientsCompareKrishnanEtAl2014.png')
ipywidgets.interact(miscellaneous.display_image, figure=figures);

#### Vorticity field

In [ ]:
simulation.plot_field_contours_visit('vorticity', (-5.0, 5.0),
                                     body='flyingSnake2dAoA25ds004',
                                     solution_folder='numericalSolution',
                                     view=(-2.0, -5.0, 15.0, 5.0),
                                     width=600)

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory,
                                     'images',
                                     'vorticity_-2.00_-5.00_15.00_5.00'))

#### Pressure field

In [ ]:
simulation.plot_field_contours_visit('pressure', (-1.0, 0.5),
                                     body='flyingSnake2dAoA25ds004',
                                     solution_folder='numericalSolution',
                                     view=(-2.0, -5.0, 15.0, 5.0),
                                     width=600)

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory,
                                     'images',
                                     'pressure_-2.00_-5.00_15.00_5.00'))